**Temat:** Analiza sentymentu w tekstach internetowych w oparciu o sieci typu Transformer

**Wprowadzenie:** Analiza sentymentu to technika przetwarzania języka naturalnego (NLP), która identyfikuje ton emocjonalny w tekście, klasyfikując go na pozytywny, negatywny lub neutralny. Wykorzystuje się ją do badania opinii klientów, monitorowania reputacji marki czy analizy treści mediów społecznościowych.

**Cel projektu:** Celem projektu jest opracowanie i implementacja modelu analizy sentymentu, który pozwoli na klasyfikację opinii użytkowników na podstawie tekstów pochodzących z Internetu. Należy przeanalizować dane tekstowe, przygotować odpowiedni model oraz zaprezentować wyniki analizy.

In [11]:
!pip3 install datasets transformers --quiet


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Ładowanie danych

In [2]:
from datasets import load_dataset

ds = load_dataset("clapAI/MultiLingualSentiment")

/Users/michalklos/Studia_Projekty/NLP/Multilingual-sentiment-analysis/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 393436/393436 [00:00<00:00, 888580.63 examples/s]


In [7]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'source', 'domain', 'language'],
        num_rows: 3147478
    })
    validation: Dataset({
        features: ['text', 'label', 'source', 'domain', 'language'],
        num_rows: 393435
    })
    test: Dataset({
        features: ['text', 'label', 'source', 'domain', 'language'],
        num_rows: 393436
    })
})


In [ ]:
# what languages are available
languages = ds['train'].unique('language')
print("Available languages:", languages)

# Create dictionary to store datasets for each language
datasets_by_language = {}

# Split train for each language (takes a while)
for lang in languages:
    datasets_by_language[lang] = ds['train'].filter(lambda x: x['language'] == lang)
    

Available languages: ['en', 'es', 'ja', 'ar', 'tr', 'fr', 'vi', 'zh', 'de', 'ru', 'ko', 'id', 'multilingual', 'pt', 'ms', 'hi', 'it']


Filter: 100%|██████████| 3147478/3147478 [00:40<00:00, 77903.16 examples/s]


In [10]:
datasets_by_language['ja'][0]

{'text': 'コードレス設計で車内の掃除もできます。\nコードレス設計で車内の掃除もできます。砂と土なども吸い込みます。掃除苦手の私でも快適に掃除ができます。',
 'label': 'positive',
 'source': 'https://huggingface.co/datasets/mteb/amazon_reviews_multi',
 'domain': 'amazon reviews',
 'language': 'ja'}

### Tokenizacja

In [12]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained("clapAI/MultiLingualSentiment")

def tokenize_and_encode(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512,  # BERT max sequence length
        return_tensors=None,  # Returns PyTorch tensors
        return_special_tokens_mask=True
    )

tokenized_datasets = {}
for lang, dataset in datasets_by_language.items():
    print(f"Tokenizing {lang} dataset...")
    tokenized_datasets[lang] = dataset.map(
        tokenize_and_encode,
        batched=True,
        remove_columns=['text', 'language']  # Remove original columns we don't need
    )
    
    # Convert label to tensor format
    tokenized_datasets[lang] = tokenized_datasets[lang].map(
        lambda x: {'labels': x['label']},
        remove_columns=['label']
    )

print("\nExample of tokenized text:")
print(tokenized_datasets[list(tokenized_datasets.keys())[0]][0])